In [1]:
from graphviz import Digraph

def trace(root):
    nodes , edges = set() , set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes , edges

def draw(root):
    dot = Digraph(format='svg' , graph_attr={'rankdir': 'LR'})

    nodes , edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid , label = "{ %s | data %.4f | grad %.4f }" % (n.label , n.data , n.grad) , shape="record")
        if n._op:
            dot.node(name = uid + n._op , label = n._op)
            dot.edge(uid + n._op , uid)
    for n1 , n2 in edges:
        dot.edge(str(id(n1)) , str(id(n2)) + n2._op)

    return dot

**Backpropogation Example**

In [2]:
import random
from backpropogation import Tensor
import numpy as np

In [3]:
w = Tensor(random.uniform(-1,1) , label='w')
b = Tensor(random.uniform(-1,1) , label='b')

In [4]:
def parameters():
    return [w , b]

In [5]:
SAMPLES = 500
x = np.arange(1 , SAMPLES)
random.shuffle(x)
y = 0.3*x + 4

x += np.random.randint(0 , 3 , size=len(x))
y += np.random.randint(0 , 3 , size=len(y))
x = (x - x.mean()) / x.std()
y = (y - y.mean()) / y.std()

split = int(SAMPLES * 0.8)

x_train = x[:split]
y_train = y[:split]
x_test = x[split:]
y_test = y[split:]


In [6]:
def get_batch(train=True):
    if train:
        idx = np.random.randint(0 , len(x_train)-1 , size = 5)
        return np.expand_dims(x_train[idx] , axis=0) , np.expand_dims(y_train[idx] , axis=0)
    
    idx = np.random.randint(0 , len(x_test)-1 , size = 5)
    return np.expand_dims(x_test[idx] , axis=0) , np.expand_dims(y_test[idx] , axis=0)

In [7]:

for epoch in range(500):

    # get batch of data
    x_batch , y_batch = get_batch()
    xs , ys = x_batch[0] , y_batch[0]

    # model(x)
    y_pred = [w * xi + b for xi in xs]
    loss = sum([(yout - ygt)**2 for ygt , yout in zip(ys,y_pred)]) # MSE

    # optimizer.zero_grad()
    for p in parameters():
        p.grad = 0.0
    loss.backward()

    # optimizer.step()
    for p in parameters():
        p.data += -0.05 * p.grad

    if epoch % 50 == 0:
        print(f"Epoch {epoch}: Loss = {loss.data:.4f} | w = {w.data:.4f} | b = {b.data:.4f}")

print(f"\nFinal Model: y = {w.data:.4f} * x + {b.data:.4f}")

Epoch 0: Loss = 5.2174 | w = 0.1169 | b = 0.6620
Epoch 50: Loss = 0.0017 | w = 0.9988 | b = 0.0108
Epoch 100: Loss = 0.0027 | w = 1.0089 | b = -0.0035
Epoch 150: Loss = 0.0022 | w = 0.9979 | b = -0.0044
Epoch 200: Loss = 0.0022 | w = 0.9968 | b = 0.0058
Epoch 250: Loss = 0.0030 | w = 0.9993 | b = -0.0039
Epoch 300: Loss = 0.0015 | w = 0.9944 | b = -0.0048
Epoch 350: Loss = 0.0020 | w = 0.9993 | b = 0.0026
Epoch 400: Loss = 0.0015 | w = 0.9993 | b = -0.0030
Epoch 450: Loss = 0.0013 | w = 0.9885 | b = -0.0020

Final Model: y = 0.9939 * x + -0.0006


In [13]:
draw(loss)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH